In [2]:
import tensorflow as tf

2023-12-14 20:20:15.942724: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-14 20:20:20.121383: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 20:20:20.121593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 20:20:20.435245: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 20:20:21.404757: I tensorflow/core/platform/cpu_feature_guar

In [3]:
model1 = tf.keras.models.load_model('./Weights') # loding the model from dir 
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26)]              0         
                                                                 
 batch_normalization (Batch  (None, 26)                104       
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 20)                540       
                                                                 
 dropout (Dropout)           (None, 20)                0         
                                                                 
 batch_normalization_1 (Bat  (None, 20)                80        
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 14)                294   